In [ ]:

!pip install -U llmcompressor transformers accelerate datasets torch

In [4]:
import torch
import transformers
import accelerate
import datasets
import llmcompressor

print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("datasets:", datasets.__version__)
print("llmcompressor:", llmcompressor.__version__)


/root/miniconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.9.1+cu128
transformers: 4.57.3
accelerate: 1.12.0
datasets: 4.4.1
llmcompressor: 0.9.0


In [ ]:
import torch
from llmcompressor import  oneshot
from llmcompressor.modifiers.awq import AWQModifier
from transformers import AutoModelForCausalLM,AutoTokenizer
from datasets import load_dataset
import random


MERGED_REPO = "acesmile/Qwen3-14B_merged" 
AWQ_DIR  = "Qwen3_14B_AWQ_Final"
DATA_PATH = "training_data_cleaned.jsonl"



def quantize_awq():


# 1. Load Tokenizer
 print(f"⏳ Đang tải Tokenizer...")
 tokenizer = AutoTokenizer.from_pretrained(MERGED_REPO, trust_remote_code=True)
    

 print("⏳ Đang nạp model...")
 model = AutoModelForCausalLM.from_pretrained(
    MERGED_REPO,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True,

)

 print(f"📂 Đang đọc dữ liệu từ: {DATA_PATH}")
 ds = load_dataset("json", data_files=DATA_PATH, split="train")

 num_samples = min(256, len(ds))
 ds = ds.shuffle(seed=42).select(range(num_samples))
    
 print(f"✅ Đã chọn {len(ds)} mẫu dữ liệu để hiệu chuẩn.") 

 def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        )
    }

 ds = ds.map(preprocess, remove_columns=ds.column_names)

 model.eval()
   
    
 recipe =[ AWQModifier(
    targets=["Linear"],
    scheme="W4A16",
    ignore=["lm_head"],
     duo_scaling=False,
)
         ]

 print("🔨 Đang thực hiện nén AWQ (Oneshot)...")

 oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=1024,
    num_calibration_samples=256,
)

 model.save_pretrained(AWQ_DIR, save_compressed=True)
 tokenizer.save_pretrained(AWQ_DIR)

 print(f"✅ Đã xong! Model AWQ nằm tại: {AWQ_DIR}")


if __name__ == "__main__":
    quantize_awq()

⏳ Đang tải Tokenizer...
⏳ Đang nạp model...


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.49s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


📂 Đang đọc dữ liệu từ: training_data_cleaned.jsonl
✅ Đã chọn 256 mẫu dữ liệu để hiệu chuẩn.
🔨 Đang thực hiện nén AWQ (Oneshot)...


Tokenizing: 100%|█████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 823.70 examples/s]

2026-01-16T12:36:35.712366+0000 | reset | INFO - Compression lifecycle reset
2026-01-16T12:36:35.714998+0000 | from_modifiers | INFO - Creating recipe from modifiers


...

Some parameters are on the meta device because they were offloaded to the cpu.
Updating global scales: 100%|█████████████████████████████████████████████████████| 280/280 [00:00<00:00, 331659.17it/s]
Fusing global scales: 887it [00:00, 331581.79it/s]
Calibrating weights: 100%|████████████████████████████████████████████████████████████| 280/280 [00:07<00:00, 37.40it/s]

2026-01-16T12:36:47.745549+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2026-01-16T12:36:48.078001+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`
2026-01-16T12:36:48.102701+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 280it [00:55,  5.08it/s]


✅ Đã xong! Model AWQ nằm tại: Qwen3_14B_AWQ_Final


In [ ]:
from huggingface_hub import HfApi, login
from config import HF_TOKEN

# ================= CẤU HÌNH =================
 
REPO_ID = "acesmile/awq_qwen3-14B_final"    
FOLDER_PATH = "Qwen3_14B_AWQ_Final"     # 
# ============================================

def simple_upload():
    print(f"🔑 Đang đăng nhập...")
    login(token=HF_TOKEN)
    
    print(f"🚀 Đang upload toàn bộ folder '{FOLDER_PATH}' lên '{REPO_ID}'...")
    print("⏳ Vui lòng chờ, không tắt trình duyệt/tab cho đến khi xong...")

    api = HfApi()
    
    
    api.upload_folder(
        folder_path=FOLDER_PATH,
        repo_id=REPO_ID,
        repo_type="model",
        commit_message="Upload model AWQ qwen3"
    )
    
    print("✅ Đã Upload thành công!")

if __name__ == "__main__":
    simple_upload()

🔑 Đang đăng nhập...
🚀 Đang upload toàn bộ folder 'Qwen3_14B_AWQ_Final' lên 'acesmile/awq_qwen3_14b_final'...
⏳ Vui lòng chờ, không tắt trình duyệt/tab cho đến khi xong...


Processing Files (0 / 0): |                                                                |  0.00B /  0.00B            
Processing Files (1 / 2):   0%|                                                            | 11.9MB / 9.94GB, 2.62MB/s  
Processing Files (1 / 3):   0%|                                                            | 18.2MB / 9.94GB, 17.0MB/s  
Processing Files (1 / 3):   0%|▎                                                           | 41.9MB / 9.94GB, 50.7MB/s  
Processing Files (1 / 3):   1%|▎                                                           | 61.3MB / 9.94GB, 62.3MB/s  
Processing Files (1 / 3):   1%|▌                                                           | 87.0MB / 9.94GB, 75.6MB/s  
Processing Files (1 / 3):   1%|▋                                                           |  110MB / 9.94GB, 82.4MB/s  
Processing Files (1 / 3):   1%|▊                                                           |  136MB / 9.94GB, 88.7MB/s  
Processing Files (1 / 3):   2%|▉

✅ Đã Upload thành công!
